In [1]:
# Import the necessary module to work with files
import os
import sys
# List the files in the dataset to verify the presence of model.py
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Add the directory containing your module to the Python path
module_path = '/kaggle/input/k19unet3d-py'  # Update this with the correct path
sys.path.append(module_path)

# Import your module
import Unet3d

/kaggle/input/k19unet3d-py/Unet3d.py
/kaggle/input/k19processeddata/kits19trainXy/case_00008train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00131train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00139train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00064train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00135train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00121train_y.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00147train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00012train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00047train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00025train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00039train_y.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00046train_y.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00112train_X.npy
/kaggle/input/k19processeddata/kits19trainXy/case_00000train_X.npy
/kaggle/input/k19processe

In [2]:
!pip install patchify

In [3]:
from Unet3d import simple_unet_model
import os
import tensorflow as tf
from skimage import io
from patchify import patchify, unpatchify
import numpy as np
from matplotlib import pyplot as plt
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import MeanIoU
import keras.optimizers as optim
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.optimizers.schedules import CosineDecay

In [4]:

 
def load_and_preprocess_data(image_path, mask_path):
    # Load and preprocess your data from file paths
    image = np.load(image_path)  # Adjust this part for your data format
    mask = np.load(mask_path)    # Adjust this part for your data format
    print("image shape=",image.shape)  
    print("mask shape=",mask.shape)
    return image, mask




def custom_datagen(image_paths, mask_paths, batch_size):

    L = len(image_paths)

    # Create a generator that yields batches of the preprocessed images and masks
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)

            # Load and preprocess the data
            X = []
            Y = []
            for i in range(batch_start, limit):
                image, mask = load_and_preprocess_data(image_paths[i], mask_paths[i])
                X.append(image)
                Y.append(mask)
                         
            X = np.array(X)
            Y = np.array(Y)
            X = X.reshape(-1, 64, 64, 64, 3)
            Y = Y.reshape(-1, 64, 64, 64, 3)
            print("X shape=",X.shape)              
            print("Y shape=",Y.shape)
           

            yield (X, Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size


# Define the range of case numbers
case_numbers = list(range(140))

# Create lists to store the file paths
train_img_paths = []
train_mask_paths = []
val_img_paths = []
val_mask_paths = []

for case_number in case_numbers:
    case_str = f'case_{case_number:05d}'
    train_img_dir = os.path.join('/kaggle/input/k19processeddata/kits19trainXy/', f'{case_str}train_X.npy')
    train_mask_dir = os.path.join('/kaggle/input/k19processeddata/kits19trainXy/', f'{case_str}train_y.npy')
    train_img_paths.append(train_img_dir)
    train_mask_paths.append(train_mask_dir)
    
# Split the data into training and validation sets
train_img_paths, val_img_paths, train_mask_paths, val_mask_paths = train_test_split(train_img_paths, train_mask_paths, test_size=0.2, random_state=42)
batch_size = 1

# Create the custom datagen generators
train_dataset = custom_datagen(train_img_paths, train_mask_paths, batch_size)
val_dataset = custom_datagen(val_img_paths, val_mask_paths, batch_size)

print(train_dataset, val_dataset)


<generator object custom_datagen at 0x7c575f7a3b50> <generator object custom_datagen at 0x7c575f7a3a00>


In [5]:

# Get a batch of data from the train_dataset object
img, msk = train_dataset.__next__()
print("disimage shape=",img.shape)  
print("dismask shape=",msk.shape)


image shape= (64, 64, 64, 64, 3)
mask shape= (64, 64, 64, 64, 3)
X shape= (64, 64, 64, 64, 3)
Y shape= (64, 64, 64, 64, 3)
disimage shape= (64, 64, 64, 64, 3)
dismask shape= (64, 64, 64, 64, 3)


In [6]:
def dice_coefficient(y_true, y_pred):
    smooth= 1e-15
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)

def dice_loss(y_true, y_pred):
    # Calculate Dice coefficient for each class and take the mean
    num_classes = 3  # Change this to the actual number of classes
    dice_coefficients = []
    for i in range(num_classes):
        class_true = y_true[..., i]
        class_pred = y_pred[..., i]
        dice_coefficients.append(dice_coefficient(class_true, class_pred))
    return 1.0 - tf.reduce_mean(dice_coefficients)


In [7]:
IOU_keras = MeanIoU(num_classes=3)  
lr = 1e-4  # Initial learning rate
total_epochs = 20  # Total number of epochs
#warmup_epochs = 5  # Number of epochs for warmup

model_path = '/kaggle/working/kits19unet_3d.hdf5'
csv_path = '/kaggle/working/kits19unet_3d.csv'

steps_per_epoch = len(train_img_paths)//batch_size
val_steps_per_epoch = len(val_img_paths)//batch_size

model = simple_unet_model(IMG_CHANNELS=3,IMG_DEPTH=64,IMG_HEIGHT=64,IMG_WIDTH=64,num_classes=3)

# Define a learning rate schedule with cosine decay and warmup
lr_schedule = CosineDecay(initial_learning_rate=lr, decay_steps= 95)

# Create an optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss=dice_loss, metrics=[dice_coefficient])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-9, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 64,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv3d_19 (Conv3D)             (None, 64, 64, 64,   1312        ['input_2[0][0]']                
                                16)                                                               
                                                                                                  
 dropout_9 (Dropout)            (None, 64, 64, 64,   0           ['conv3d_19[0][0]']              
                                16)                                                         

In [ ]:
# Train your model
model.fit(train_dataset, epochs=total_epochs,steps_per_epoch=steps_per_epoch, validation_data=val_dataset,validation_steps=val_steps_per_epoch, callbacks=callbacks)

model.save(model_path)

image shape= (64, 64, 64, 64, 3)
mask shape= (64, 64, 64, 64, 3)
X shape= (64, 64, 64, 64, 3)
Y shape= (64, 64, 64, 64, 3)
Epoch 1/20
image shape= (64, 64, 64, 64, 3)
mask shape= (64, 64, 64, 64, 3)
X shape= (64, 64, 64, 64, 3)
Y shape= (64, 64, 64, 64, 3)
  1/112 [..............................] - ETA: 3:55:34 - loss: 0.8344 - dice_coefficient: 0.3070image shape= (64, 64, 64, 64, 3)
mask shape= (64, 64, 64, 64, 3)
X shape= (64, 64, 64, 64, 3)
Y shape= (64, 64, 64, 64, 3)
  2/112 [..............................] - ETA: 4:06:58 - loss: 0.8312 - dice_coefficient: 0.3162image shape= (64, 64, 64, 64, 3)
mask shape= (64, 64, 64, 64, 3)
X shape= (64, 64, 64, 64, 3)
Y shape= (64, 64, 64, 64, 3)
  3/112 [..............................] - ETA: 3:56:14 - loss: 0.8284 - dice_coefficient: 0.3215image shape= (64, 64, 64, 64, 3)
mask shape= (64, 64, 64, 64, 3)
X shape= (64, 64, 64, 64, 3)
Y shape= (64, 64, 64, 64, 3)
  4/112 [>.............................] - ETA: 3:49:31 - loss: 0.8224 - dice_coeff